In [1]:
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.exc import OperationalError
import os
from dotenv import load_dotenv
import psycopg2

# Carregar o arquivo Excel em um DataFrame
dados = pd.read_excel('dados/Relatório Consolidado.xlsx', sheet_name="recebimento")

In [2]:
# Carregar variáveis do arquivo .env
load_dotenv()

# Obter as credenciais do banco de dados do .env
db_host = os.getenv("DB_HOST")
db_port = os.getenv("DB_PORT")
db_name = os.getenv("DB_NAME")
db_user = os.getenv("DB_USER")
db_password = os.getenv("DB_PASSWORD")

In [3]:
dados.head()

,DATA,CATEGORIA,CLIENTE,ITEM_PCG,TOTAL_RECEBIDO
0,2024-08-01,TELEMEDICINA,PagSmart Pagamentos,TELEMEDICINA VAREJO,3100.50
1,2024-08-03,SEGURO,ProtegeMais Seguros,SEGURO CONDOMINIAL,2650.75
2,2024-08-05,TELEMEDICINA,MediBridge,TELEMEDICINA CORPORATIVO,4800.00
3,2024-08-07,PLANO DE SAÚDE,VidaTotal Saúde,PLANO DE SAÚDE PJ (+ 30 VIDAS),3920.00
4,2024-08-08,SEGURO,HDS Proteção,SEGURO CONDOMINIAL,1245.90


In [4]:
dados.tail()

,DATA,CATEGORIA,CLIENTE,ITEM_PCG,TOTAL_RECEBIDO
493,2024-01-04,TELEMEDICINA,PagSmart Pagamentos,TELEMEDICINA VAREJO,186.74
494,2024-01-03,TELEMEDICINA,PagSmart Pagamentos,TELEMEDICINA VAREJO,243.89
495,2024-01-02,TELEMEDICINA,PagSmart Pagamentos,TELEMEDICINA VAREJO,473.42
496,2024-01-02,SEGURO,SimPro Seguros,SEGURO CONDOMINIAL,95.24
497,2024-01-02,SEGURO,TProtege,SEGURO CONDOMINIAL,4.86


In [5]:
dados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 498 entries, 0 to 497
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   DATA            498 non-null    datetime64[ns]
 1   CATEGORIA       498 non-null    object        
 2   CLIENTE         498 non-null    object        
 3   ITEM_PCG        498 non-null    object        
 4   TOTAL_RECEBIDO  498 non-null    float64       
dtypes: datetime64[ns](1), float64(1), object(3)
memory usage: 19.6+ KB


In [6]:
# Criar a string de conexão usando as variáveis de ambiente
postgres_str = f'postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}'
engine = create_engine(postgres_str)

try:
    with engine.connect() as connection:
        # Criar o esquema 'telemedicina' se não existir
        connection.execute("CREATE SCHEMA IF NOT EXISTS telemedicina;")
        print("Esquema 'telemedicina' criado com sucesso!")

        # Nome do esquema e da tabela no banco de dados
        schema_name = 'telemedicina'
        table_name = 'recebimentos'

        # Criar a tabela no banco de dados
        create_table_query = f"""
        CREATE TABLE IF NOT EXISTS {schema_name}.{table_name} (
            DATA DATE,
            CATEGORIA VARCHAR(255),
            CLIENTE VARCHAR(255),
            ITEM_PCG VARCHAR(255),
            TOTAL_RECEBIDO FLOAT,
            STATUS VARCHAR(10)
        );
        """
        connection.execute(create_table_query)
        print(f"Tabela {schema_name}.{table_name} criada com sucesso!")

        # Inserir os dados no banco de dados, substituindo a tabela existente
        dados.to_sql(table_name, engine, schema=schema_name, if_exists='replace', index=False)
        print("Dados substituídos no banco de dados com sucesso!")

except OperationalError as oe:
    print(f"Erro de conexão com o banco de dados: {oe}")
except Exception as e:
    print(f"Erro ao criar a tabela ou inserir dados no banco de dados: {e}")

Esquema 'telemedicina' criado com sucesso!
Tabela telemedicina.recebimentos criada com sucesso!
Dados substituídos no banco de dados com sucesso!
